In [1]:
import os
import numpy as np
import time, os, copy, datetime, glob
from tqdm import tqdm
from skimage import io
from PIL import Image
Image.MAX_IMAGE_PIXELS = None
# PRE_PATH = '/BrainSeg/Classify_Results/GMBData/'
# pred_names = os.listdir(PRE_PATH)
# print(len(pred_names))

## Generate Mask for comparison

In [2]:
# Helper function to convert PyVips Image to numpy array
# map vips formats to np dtypes
format_to_dtype = {
    'uchar': np.uint8,
    'char': np.int8,
    'ushort': np.uint16,
    'short': np.int16,
    'uint': np.uint32,
    'int': np.int32,
    'float': np.float32,
    'double': np.float64,
    'complex': np.complex64,
    'dpcomplex': np.complex128,
}

# vips image to numpy array
def vips2numpy(vi):
    return np.ndarray(buffer=vi.write_to_memory(),
                      dtype=format_to_dtype[vi.format],
                      shape=[vi.height, vi.width, vi.bands])

def img_frombytes(data):
    size = data.shape[::-1]
    databytes = np.packbits(data, axis=1)
    return Image.frombytes(mode='1', size=size, data=databytes)

In [3]:
# Specify images to run
IMG_DIR = '/BrainSeg/norm_png/'
SAVE_DATA_DIR = '/BrainSeg/Classify_Results/ResNet_Aug/GMBData/'
SAVE_IMG_DIR = '/BrainSeg/Classify_Results/ResNet_Aug/GMBMasks/'

filenames = glob.glob(IMG_DIR + '*.png')
filenames = [filename.split('/')[-1] for filename in filenames]
filenames = sorted(filenames)
total_image_num = len(filenames)

idx_image_to_eval = slice(0,30)      # Modify this line to select images to eval
filenames = filenames[idx_image_to_eval]
print('Run for the following %d images out of %d images:\n' % (len(filenames), total_image_num), filenames)


Run for the following 30 images out of 30 images:
 ['NA3777-02_AB.png', 'NA4077-02_AB.png', 'NA4092-02_AB.png', 'NA4107-02_AB.png', 'NA4160-02_AB.png', 'NA4195-02_AB.png', 'NA4256-02_AB.png', 'NA4299-02_AB.png', 'NA4391-02_AB.png', 'NA4450-02_AB.png', 'NA4463-02_AB.png', 'NA4471-02_AB.png', 'NA4553-02_AB.png', 'NA4626-02_AB.png', 'NA4672-02_AB.png', 'NA4675-02_AB.png', 'NA4691-02_AB.png', 'NA4695-02_AB.png', 'NA4894-02_AB17-24.png', 'NA4907-02_AB17-24.png', 'NA4944-02_AB17-24.png', 'NA4945-02_AB17-24.png', 'NA4967-02_AB17-24.png', 'NA4971-02_AB17-24.png', 'NA4972-02_AB17-24.png', 'NA4992-02_AB17-24.png', 'NA4993-02_AB17-24.png', 'NA5010-02_AB17-24.png', 'NA5015-02_AB17-24.png', 'NA5029-02_AB17-24.png']


__GM and WM__

In [4]:
# Start evaluating
t = tqdm(total=len(filenames))
for filename in filenames:
    t.set_description_str("Image " + filename + ": ")
    t.refresh()
    t.write("", end=' ')

    # Load data
    data = np.load(SAVE_DATA_DIR + filename.split('.')[-2] + '.npy')

    # Create grey white arrays
    grey_arr = np.zeros_like(data, dtype='bool')
    white_arr = np.zeros_like(data, dtype='bool')

    # Set white matter to 1 for white_arr
    white_arr[data == 1] = True
    # Set grey matter to 2 for grey_arr
    grey_arr[data == 2] = True
    

    # Create save images
    save_img_grey = img_frombytes(grey_arr)
    save_img_white = img_frombytes(white_arr)

    # Save the images
    save_img_grey.save(SAVE_IMG_DIR + filename.split('.')[-2] + '-Grey.png')
    save_img_white.save(SAVE_IMG_DIR + filename.split('.')[-2] + '-White.png')

    t.update()
t.close()

Image NA3777-02_AB.png:   0%|          | 0/30 [00:00<?, ?it/s]

Image NA5029-02_AB17-24.png: 100%|██████████| 30/30 [00:00<00:00, 50.73it/s]

__Background__

In [5]:
# Start evaluating
t = tqdm(total=len(filenames))
for filename in filenames:
    t.set_description_str("Image " + filename + ": ")
    t.refresh()
    t.write("", end=' ')

    # Load data
    data = np.load(SAVE_DATA_DIR + filename.split('.')[-2] + '.npy')

    # Create background array
    back_arr = np.zeros_like(data, dtype='bool')

    # Set background to 1 for back_arr
    back_arr[data == 0] = True

    # Create save image
    save_img_back = img_frombytes(back_arr)

    # Save the image
    save_img_back.save(SAVE_IMG_DIR + filename.split('.')[-2] + '-Background.png')

    t.update()
t.close()

Image NA5029-02_AB17-24.png: 100%|██████████| 30/30 [00:00<00:00, 211.10it/s]

## Accuracy